# 🚀**Taret 2035 Aircheck DEL-ML example**🧬
Welcome to the Target 2035 Aircheck DEL-ML Model Training and Prediction example Notebook!

This notebook will walk you through the basic steps involved in training, evaluating, and screening small molecules using machine learning models built on chemical fingerprints.

**This is a sample workflow that illustrates how selections can be made and compounds clustered based on their chemical diversity. This very simple pipeline is using exclusively the training set, which is not best practice and should not be repeated. The goal is only to provide sample scripts that can be adapted for this challenge.**

## Loading input file

Here we use the data in the file *WDR91.parquet* as both training and external test dataset.

In an ideal case the external test set should not be selected at random from the existing training data, but here it is done for the sake of simplicity.

The file containing the training set file can be downloaded from [Aircheck](https://aircheck.ai/datasets), from the DEL-Datasets, target WDR91, and partner HitGen.

The file contains multiple columns containing important data regarding the DEL library and the compounds (BB1_ID, BB2_ID, NTC_VALUE, ENRICHMENT, etc.), the fingerprints used to represent the molecules (ECFP4, ECFP6, FCFP4, FCFP6, TOPTOR, MACCS, RDK, AVALON), and  the label LABEL identifying the molecule as a DEL hit (**1**) or not (**0**).

In this notebook only the columns ECFP4 and LABEL are read from the file.

To simulate a virtual screening campaign the dataset is split in two:
* Training set: used to train the model.
* Test set: used to evaluate the performance of the model in a virtual screening-like scenario.

**It is not best practice to perform random train-test split on training set data, since similar molecules could be present in both the training and test set, leading to overoptimistic results.**

In [ ]:
import sys
sys.path.append("../ReadingParquetFiles") #Used to import module from different folder in the repository 
from read_parquet_utils import read_parquet_file
from sklearn.model_selection import train_test_split
import numpy as np

data_file = "WDR91.parquet" #Change to the path to your file
df = read_parquet_file(data_file, columns = ['ECFP4', 'LABEL'])

x_train, x_test, y_train, y_test = train_test_split(df.ECFP4,
                                                    df.LABEL,
                                                    test_size=0.10,
                                                    random_state=42,
                                                    shuffle=True,
                                                    stratify=df.LABEL,)
x_train = np.stack(x_train)
x_test = np.stack(x_test)
y_train = y_train.to_numpy(dtype=int)
y_test = y_test.to_numpy(dtype=int)

## Model training
A random forest classification model was trained on count ECFP4. The model is kept at default hyperparamters except for the number of estimators, reduced from 100 to 10 to speed up calculations.

### Cross validation
The performance of the trained models are evaluated in 5-fold cross validation.

The training set is split in five subsets, five different models are trained using four of the five subsets, while the one not used for training is used for evaluation.

Cross validation gives an estimate of the performance of a trained model, before using it on the test set.

Here the models are evaluated based on different metrics, with a value of 1 in each of them representing an ideal model:
* **Precision**: number of molecules predicted as active which are active.
* **Recall**: fraction of active molecules predicted as active.
* **F1-score**: harmonic mean of precision and recall.
* **AUC-ROC**: measures the ability of the score in prioritizing active molecules (higher scores) against inactive molecules (lower scores).

In [2]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score,roc_auc_score

skf = StratifiedKFold(n_splits=5, shuffle = True, random_state=42)
for i, (train_index, test_index) in enumerate(skf.split(x_train, y_train)):
    model = RandomForestClassifier(n_estimators=10) #Reduced from the defult 100 to save time
    model.fit(x_train[train_index], y_train[train_index])
    y_score = model.predict_proba(x_train[test_index])[:,1]
    y_pred = model.predict(x_train[test_index])

    metrics = {"Precision": precision_score(y_train[test_index], y_pred, zero_division=0),
                "Recall": recall_score(y_train[test_index], y_pred),
                "F1-Score": f1_score(y_train[test_index], y_pred),
                "AUC-ROC": roc_auc_score(y_train[test_index], y_score)}

    print(f"Fold {i+1} Metrics:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")
    print("-" * 40)


Fold 1 Metrics:
Precision: 0.8869
Recall: 0.6295
F1-Score: 0.7364
AUC-ROC: 0.9487
----------------------------------------
Fold 2 Metrics:
Precision: 0.8874
Recall: 0.6315
F1-Score: 0.7379
AUC-ROC: 0.9527
----------------------------------------
Fold 3 Metrics:
Precision: 0.8864
Recall: 0.6309
F1-Score: 0.7371
AUC-ROC: 0.9508
----------------------------------------
Fold 4 Metrics:
Precision: 0.8851
Recall: 0.6288
F1-Score: 0.7352
AUC-ROC: 0.9486
----------------------------------------
Fold 5 Metrics:
Precision: 0.8888
Recall: 0.6355
F1-Score: 0.7411
AUC-ROC: 0.9509
----------------------------------------


### Evaluation on the "external" test set

In [3]:
model = RandomForestClassifier(n_estimators=10).fit(x_train, y_train)
y_score = model.predict_proba(x_test)[:,1]
y_pred = model.predict(x_test)

metrics = {"Precision": precision_score(y_test, y_pred, zero_division=0),
                "Recall": recall_score(y_test, y_pred),
                "F1-Score": f1_score(y_test, y_pred),
                "AUC-ROC": roc_auc_score(y_test, y_score)}

print(f"External test set Metrics:")
for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")


External test set Metrics:
Precision: 0.8905
Recall: 0.6442
F1-Score: 0.7476
AUC-ROC: 0.9545


### Saving the model
The model can be saved as a *pickle* file for future use or for sharing

In [4]:
import pickle
with open('Aircheck_WDR91_model.pkl', 'wb') as out:
    pickle.dump(model, out)

## Molecule selection based on chemical diversity

A machine learning model can identify thousands of potentially active molecules in a chemical library, however in drug discovery only a limited number of them can be tested experimentally (10-100 molecules).

To better explore the chemical space, high-ranking molecules are often clustered based on molecular similarity to select a smaller number of diverse compounds.

Here the top 5000 molecules were clustered based on their Tanimoto similarity (also known as Jaccard similarity/distance) between binary ECFP4 using the agglomerative clustering algorithm, selecting a diverse set of 200 and 500 molecules. From each cluster the molecule with the highest probability score was selected.

**Here agglomerative clustering is used as an example, there are other clustering algorithms or diveristy selection methods (eg. Butina clustering, LeaderPicker from the rdkit) which should be considered.**

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import pdist, squareform

top_5000 = np.argsort(y_score)[-1:-5001:-1]
cluster_fp = x_test[top_5000]
distance = pdist(cluster_fp != 0, metric = 'jaccard') #Turn the count ECFP4 into binary ECFP4
clustering_200 = AgglomerativeClustering(n_clusters=200,
                                    metric='precomputed',
                                    linkage='complete')
cluster_labels_200 = clustering_200.fit_predict(squareform(distance))
clustering_500 = AgglomerativeClustering(n_clusters=500,
                                    metric='precomputed',
                                    linkage='complete')
cluster_labels_500 = clustering_500.fit_predict(squareform(distance))

In [ ]:
hits_200 = 0
hits_500 = 0
seen_cluster_200 = set()
seen_cluster_500 = set()
for sorted_index, original_index in enumerate(top_5000):
    if cluster_labels_200[sorted_index] not in seen_cluster_200:
        seen_cluster_200.add(cluster_labels_200[sorted_index])
        if y_test[original_index]:
            hits_200 += 1
    if cluster_labels_500[sorted_index] not in seen_cluster_500:
        seen_cluster_500.add(cluster_labels_500[sorted_index])
        if y_test[original_index]:
            hits_500 += 1
print(f"Numer or hits found in the selected 200 molecules: {hits_200}")
print(f"Numer or hits found in the selected 500 molecules: {hits_500}")

Numer or hits found in the selected 200 molecules: 133
Numer or hits found in the selected 500 molecules: 241
